# Calm Data Analysis

___
![Calm Logo](https://www.insightpartners.com//assets/media/2018/06/Calm-e1568058016375.png)

> **Assignment:** We have 5 tabs of spend data from different sample media sources Channels A, B, C, D and E as well as a tab with sample Calm attribution data which shows the total number of Installs, Trials, Purchases etc for a given media source by date, platform and country. It is important as a member of the UA team to be able to analyze this type of data to get a read on how specific channels/geos are performing in order to make campaign optimizations.

## Table of Content

### [Import](#import)
### [Explore Dataset](#explore)
- **[Explore Spend Dataset](#spend_explore)**
- **[Explore Backend Dataset](#backend_explore)**

### [Clean Dataset](#clean)
- **[Clean Spend Dataset](#spend_clean)**
- **[Clean Backend Dataset](#backend_clean)**

### [Merge Datasets](#merge)

### [Table 1](#t1)

- **[Question 1](#q1)**
- **[Question 2](#q2)**
- **[Question 3](#q3)**
- **[Question 4](#q4)**
- **[Question 5](#q5)**
- **[Question 6](#q6)**


### [Table 2](#t2)

- **[Question 7](#q7)**
- **[Question 8](#q8)**
- **[Question 9](#q9)**

## Import Data <a id='import'></a>


In the section below I import the workbook with the data described in the assignment above. I'll also be importing some libraries required to conduct the analysis.

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
raw_df_dict = pd.read_excel(
    './raw_data/Marketing Ops Take-Home vff.xlsx', 
    sheet_name=None, 
    na_values=['','#N/A'], 
    keep_default_na=False
)

## Explore Datasets <a id='explore'></a>

### Explore Spend Dataset <a id='spend_explore'></a>

I import the data and merge all of the channel tabs into one dataset so it will be easier to explore, clean and aggregate. Below I see there are 1077 rows and 7 columns in the spend dataset. I also noticed that the Country Code column has 2 missing value.

In [2]:
list_of_spend_df = [raw_df_dict[key] for key in raw_df_dict.keys() if key not in 'Backend data']
spend_df = pd.concat(list_of_spend_df, ignore_index=True)
spend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1077 entries, 0 to 1076
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1077 non-null   datetime64[ns]
 1   Region        1077 non-null   object        
 2   Platform      1077 non-null   object        
 3   Country Code  1075 non-null   object        
 4   Spend         1077 non-null   float64       
 5   Media Source  1077 non-null   object        
 6   Traffic Type  1077 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 59.0+ KB


Here are the 2 rows that are missing country codes. For the questions I'll be answering in the analysis this won't affect me but I've noted it to be investigated and potentially reduce the amount of missing country codes in the future. 

In [3]:
spend_df[spend_df['Country Code'].isna()]

,Date,Region,Platform,Country Code,Spend,Media Source,Traffic Type
112,2022-05-01,Non-US,ios,NaN,187.64,Channel A,Paid
374,2022-05-01,Non-US,android,NaN,14.27,Channel A,Paid


If I look at all the unique values of the Date column I can see that all rows share the same date, 2022-05-01. For the purposes of readability I will add a month column with the value of May.

In [4]:
print(spend_df['Date'].unique())

['2022-05-01T00:00:00.000000000']


### Explore Backend Dataset <a id='backend_explore'></a>

Below I explore the backend dataset. I noticed that there's 8 values missing in the country column. Additionally, about 54% of data is missing for the forecasted_purchasers, ltv and forcasted_ltv_rev columns.

In [5]:
backend_df = raw_df_dict['Backend data']
backend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562 entries, 0 to 1561
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   month                  1562 non-null   datetime64[ns]
 1   platform               1562 non-null   object        
 2   media_source           1562 non-null   object        
 3   traffic_type           1562 non-null   object        
 4   country                1554 non-null   object        
 5   installers             1562 non-null   float64       
 6   trial_starts           1562 non-null   float64       
 7   forecasted_purchasers  839 non-null    float64       
 8   ltv                    839 non-null    float64       
 9   forcasted_ltv_rev      839 non-null    float64       
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 122.2+ KB


<a id='backend_stats'></a>
Here are some descriptive statistics for all the numerical columns in the backend data. Keep in mind this is still including the rows with missing data.

In [6]:
backend_df.describe().round(2)

,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev
count,1562.00,1562.00,839.00,839.00,839.00
mean,1579.59,141.60,137.65,65.61,10519.11
std,10236.40,1174.77,985.24,14.45,88707.95
min,0.00,0.00,1.09,32.54,35.92
25%,11.00,0.00,1.16,55.71,103.93
50%,63.00,2.00,4.59,63.69,316.50
75%,383.75,19.00,27.05,73.28,1682.07
max,284978.36,33356.98,22756.81,130.48,2140966.95


Here I filter for just the rows with missing data in the three columns I identified earlier for the backend dataset. I can see that 723 rows are missing consistently within the same rows.

In [7]:
backend_df[
    backend_df['forecasted_purchasers'].isna()
    & backend_df['ltv'].isna()
    & backend_df['forcasted_ltv_rev'].isna()
].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 723 entries, 179 to 1561
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   month                  723 non-null    datetime64[ns]
 1   platform               723 non-null    object        
 2   media_source           723 non-null    object        
 3   traffic_type           723 non-null    object        
 4   country                715 non-null    object        
 5   installers             723 non-null    float64       
 6   trial_starts           723 non-null    float64       
 7   forecasted_purchasers  0 non-null      float64       
 8   ltv                    0 non-null      float64       
 9   forcasted_ltv_rev      0 non-null      float64       
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 62.1+ KB


Using the same descriptive statistics as above for just the rows with missing data I can see that for trial_starts the max is 31 and the mean is .93. Both of these value are much lower compared to what we were seeing for the complete dataset [here](#backend_stats).

Since we're seeing an average of .9 trial_starts then we can expect the forecasted_purchasers to be less then .9. At this point I'm comfortable enough to assume that all the missing values are actually 0 and I will fill in the missing values with 0 when I clean the dataset.

In [8]:
backend_df[backend_df['forecasted_purchasers'].isna()].describe()

,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev
count,723.000000,723.000000,0.0,0.0,0.0
mean,48.875519,0.934993,NaN,NaN,NaN
std,117.916779,2.267421,NaN,NaN,NaN
min,0.000000,0.000000,NaN,NaN,NaN
25%,3.000000,0.000000,NaN,NaN,NaN
50%,12.000000,0.000000,NaN,NaN,NaN
75%,42.000000,1.000000,NaN,NaN,NaN
max,1428.000000,31.000000,NaN,NaN,NaN


Similar to the source spend dataset all rows in the backend dataset share the same date, 2022-05-01. Since the month value is the same across both datasets this isn't a factor I need to consider but in order to satisfy the assignment instructions I will leave it in when I create the tables. I will also convert the month column to May when I go to clean both datasets in the next section.

In [9]:
print(backend_df['month'].unique())

['2022-05-01T00:00:00.000000000']


## Clean Datasets <a id='clean'></a>

After exploring our dataset we've identified a couple of items that need to be cleaned before we can to aggregate our data into the table 1 and 2.

### Clean Spend Dataset <a id='spend_clean'></a>

For the Spend dataset I'll first add the month column for improved readability. I will also convert the header naming conventions to match that of the backend dataset. Lastly I will convert traffic_type and platform values to lowercase the traffic_type to be consistent with backend dataset. Below you'll see just the first 5 rows for the cleaned spend dataset.

In [10]:
# add month columns
spend_df['month'] = spend_df['Date'].dt.month_name()

# change header naming convention
spend_df.columns = spend_df.columns.str.lower().str.replace(' ', '_')

# rename country_code to country so that it matches backend dataset 
spend_df.rename(
    columns={'country_code': 'country'},
    inplace=True)

# convert values in 
lower_val_cols = ['traffic_type', 'platform']
spend_df[lower_val_cols] = spend_df[lower_val_cols].apply(lambda ser: ser.str.lower())

# show first 5 rows
spend_df.head().round(2)

,date,region,platform,country,spend,media_source,traffic_type,month
0,2022-05-01,US,ios,US,1085707.44,Channel A,paid,May
1,2022-05-01,Non-US,ios,UK,261301.81,Channel A,paid,May
2,2022-05-01,Non-US,ios,BR,109037.53,Channel A,paid,May
3,2022-05-01,Non-US,ios,DE,86208.38,Channel A,paid,May
4,2022-05-01,Non-US,ios,CA,72643.95,Channel A,paid,May


### Cleaning Backend Data  <a id='backend_clean'></a>

I'll replace the missing value in the forecasted_purchasers, ltv and forcasted_ltv_rev columns. Then similar to cleaning the spend dataset I'll also add the month column. Lastly in order to be consistent with the spend dataset I'm going to add a region column.

In [11]:
# fill in numeric columns with missing data with 0
missing_data_columns = ['forecasted_purchasers', 'ltv', 'forcasted_ltv_rev']
backend_df[missing_data_columns] = backend_df[missing_data_columns].fillna(0)

# add month columns
backend_df['month'] = backend_df['month'].dt.month_name()

# add region column
backend_df['region'] = np.where(backend_df['country']=='US', 'US', 'Non-US')

# show first 5 rows
backend_df.head().round(2)

,month,platform,media_source,traffic_type,country,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,region
0,May,ios,organic,organic,US,284978.36,33356.98,22756.81,94.08,2140966.95,US
1,May,ios,Channel A,paid,US,123626.11,20413.25,9750.45,91.74,894491.56,US
2,May,android,organic,organic,US,113182.59,6160.72,3894.61,97.36,379188.55,US
3,May,android,Channel B,paid,US,99640.50,6325.80,2566.61,95.70,245624.38,US
4,May,ios,Channel C,paid,US,75110.22,11457.35,7583.08,94.72,718268.09,US


## Merge Datasets  <a id='merge'></a>

At this point both datasets are ready to merge together. Table 1 and 2 will stem from this merged dataset but they will be aggregated further once I get there. You'll notice I've added geographic information in the following columns full_country_name, region_specific and sub-region using country code mapping file I found online. Below you'll see a couple of rows of the merged dataset. 

I have saved this dataset along with table and table 1 in the file I shared with you called "all_datasets.xlsx".

In [31]:
# merge datasets
merged_df = spend_df.merge(
    right=backend_df,
    how='outer',
    on=['month', 'platform', 'media_source', 'region', 'country', 'traffic_type'],
    validate='one_to_one',
    indicator=True
)

# add full country name
country_df = pd.read_csv('./raw_data/country_dataset.csv', encoding='latin-1')

# filter country_df for only columns we want to mrege
country_df = country_df[['country', 'full_country_name', 'region', 'sub-region']]

merged_df = merged_df.merge(
    right=country_df,
    how='left',
    on='country',
    suffixes=('_og', '_specific'),
    validate='many_to_one',
)

# show first 5 rows
merged_df.round(2)

,date,region_og,platform,country,spend,media_source,traffic_type,month,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,_merge,full_country_name,region_specific,sub-region
0,2022-05-01,US,ios,US,1085707.44,Channel A,paid,May,123626.11,20413.25,9750.45,91.74,894491.56,both,United States of America,Americas,Northern America
1,2022-05-01,Non-US,ios,UK,261301.81,Channel A,paid,May,36083.00,7520.00,4141.03,50.79,210331.04,both,United Kingdom of Great Britain,Europe,Northern Europe
2,2022-05-01,Non-US,ios,BR,109037.53,Channel A,paid,May,57907.00,4452.00,1804.75,50.86,91782.69,both,Brazil,Americas,Latin America and the Caribbean
3,2022-05-01,Non-US,ios,DE,86208.38,Channel A,paid,May,16481.00,3297.00,1398.54,54.11,75668.22,both,Germany,Europe,Western Europe
4,2022-05-01,Non-US,ios,CA,72643.95,Channel A,paid,May,10804.00,1783.00,860.37,88.55,76183.27,both,Canada,Americas,Northern America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,NaT,Non-US,ios,WF,NaN,organic,organic,May,1.00,0.00,0.00,0.00,0.00,right_only,Wallis and Futuna,Oceania,Polynesia
1675,NaT,Non-US,android,CK,NaN,organic,organic,May,1.00,0.00,0.00,0.00,0.00,right_only,Cook Islands,Oceania,Polynesia
1676,NaT,Non-US,android,EU,NaN,organic,organic,May,1.00,0.00,0.00,0.00,0.00,right_only,multiple,Europe,Europe
1677,NaT,Non-US,ios,IO,NaN,organic,organic,May,1.00,0.00,0.00,0.00,0.00,right_only,British Indian Ocean Territory,Africa,Sub-Saharan Africa


## Table 1 <a id='t1'></a>

Now that I have better understanding of the datasets I'm working with and I've addressed any data integrity issues via cleaning, we can get to the fun part. Below is the first 5 rows of Table 1 as outlined in the instructions sorted by ascending spend. Please refer to the "table_1_dataset" tab in the "all_datasets.xlsx" file I shared with you to see table 1 in its entirety.

In [36]:
# define table_one_df as merged_df before manipulating further
table_one_df = merged_df

# remove organic media channel
table_one_df = table_one_df[table_one_df['media_source'] != 'organic']

# filter for relevant headers
table_one_df = table_one_df[['month', 'platform', 'media_source', 'country', 'spend', 'installers',
       'trial_starts', 'forecasted_purchasers', 'ltv', 'forcasted_ltv_rev']]

# create function to add calculated fields
def t1_add_calulated_fields(df):
    df['ltv'] = df['forcasted_ltv_rev']/df['forecasted_purchasers']
    df['cpi'] = df['spend'] / df['installers']
    df['cpt'] = df['spend'] / df['trial_starts']
    df['paid_cac'] = df['spend'] / df['forecasted_purchasers']
    df['paid_ltv/cac'] = df['ltv'] / df['paid_cac']
    df['install_to_trial_rate'] = df['trial_starts'] / df['installers']
    df['trial_to_purchase_rate'] = df['forecasted_purchasers'] / df['trial_starts']
    return df

# add calculated fields
table_one_df = t1_add_calulated_fields(table_one_df)

# sort dataset by spend
table_one_df = table_one_df.sort_values('spend', ascending=False, ignore_index=True)

# show first 5 rows
table_one_df.head().round(2)

,month,platform,media_source,country,spend,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,cpi,cpt,paid_cac,paid_ltv/cac,install_to_trial_rate,trial_to_purchase_rate
0,May,ios,Channel A,US,1085707.44,123626.11,20413.25,9750.45,91.74,894491.56,8.78,53.19,111.35,0.82,0.17,0.48
1,May,ios,Channel B,US,357251.25,67884.40,8703.50,3262.79,90.40,294966.50,5.26,41.05,109.49,0.83,0.13,0.37
2,May,ios,Channel A,UK,261301.81,36083.00,7520.00,4141.03,50.79,210331.04,7.24,34.75,63.10,0.80,0.21,0.55
3,May,android,Channel A,US,256957.04,74949.27,7170.67,2929.07,93.83,274831.14,3.43,35.83,87.73,1.07,0.10,0.41
4,May,android,Channel B,US,244877.05,99640.50,6325.80,2566.61,95.70,245624.38,2.46,38.71,95.41,1.00,0.06,0.41


## Questions 1 <a id='q1'></a>

**To achieve a 1.0 LTV/CAC, what is the cost per purchase that Calm could afford in the US on Channel A?**

In order to achieve 1.0 LTV/CAC or break-even Calm could afford up to \\$91.73 for ios and \\$93.83 for android. These are value pulled from the LTV column. For both platforms combined Calm can afford \\$92.22 per new user. Paying more then these values will put the LTV/CAC below 1.0.

**What would be the target cost per trial assuming the current trial to purchase rate?**

I've created an algorithm below that is similar to [Excels Goal Seek](https://support.microsoft.com/en-us/office/use-goal-seek-to-find-the-result-you-want-by-adjusting-an-input-value-320cb99e-f4a4-417f-b1c3-4f369d6e66c7) tool to find a value by adjusting one input. In this case the input is CPT and we need paid_ltv/cac to equal 1.0. Here are the target CPT results:
- ios Target CPT: \\$43.82
- android Target CPT: \\$38.33
- weighted Target CPT: \\$42.39


In [14]:
# filter for US and Channel A
q1_filter = table_one_df[
    (table_one_df['country']=='US')
    & (table_one_df['media_source']=='Channel A')]

# create functionn for goal seek algorithm
def cpt_goal_seek(ltv, trial_to_purchase, target_ltv_cac, level_of_accuracy):
    cpt_lower = 0
    cpt_upper = 100
    cpt = (cpt_lower + cpt_upper)/2
    cpp = cpt / trial_to_purchase
    ltv_cac = ltv / cpp
    while round(ltv_cac, level_of_accuracy) != round(target_ltv_cac, level_of_accuracy):
        if ltv_cac > target_ltv_cac:
            cpt_lower = cpt
            cpt = (cpt_lower + cpt_upper)/2
        elif ltv_cac < target_ltv_cac:
            cpt_upper = cpt
            cpt = (cpt_lower + cpt_upper) / 2
        cpp = cpt / trial_to_purchase
        ltv_cac = ltv / cpp
    return cpt

# get target cpt to get 1 paid_ltv/cac
ios_cpt = round(cpt_goal_seek(91.738513, 0.477653, 1, 5), 2)
android_cpt = round(cpt_goal_seek(93.828890, 0.408479, 1, 5), 2)
weighted_cpt = round(cpt_goal_seek(92.221407, 0.45967, 1, 5), 2)
print(f'ios: {ios_cpt}')
print(f'android: {android_cpt}')
print(f'weighted: {weighted_cpt}')

q1_filter[['country', 'media_source', 'platform', 'spend', 'trial_starts', 'ltv', 'cpt', 'paid_cac', 'paid_ltv/cac', 'trial_to_purchase_rate']].round(2)

ios: 43.82
android: 38.33
weighted: 42.39


,country,media_source,platform,spend,trial_starts,ltv,cpt,paid_cac,paid_ltv/cac,trial_to_purchase_rate
0,US,Channel A,ios,1085707.44,20413.25,91.74,53.19,111.35,0.82,0.48
3,US,Channel A,android,256957.04,7170.67,93.83,35.83,87.73,1.07,0.41


In [35]:
q1_filter_grp = t1_add_calulated_fields(q1_filter.groupby(['media_source', 'country']).sum())
q1_filter_grp[['spend', 'trial_starts', 'ltv', 'cpt', 'paid_cac', 'paid_ltv/cac', 'trial_to_purchase_rate']].round(2)

,,spend,trial_starts,ltv,cpt,paid_cac,paid_ltv/cac,trial_to_purchase_rate
media_source,country,,,,,,,
Channel A,US,1342664.48,27583.92,92.22,48.68,105.89,0.87,0.46


## Questions 2 <a id='q2'></a>

**Would you recommend that a Channel B media buyer spend increase investment in Germany (DE) or the UK next month, given May 2022 performance? What factors should they consider?**

Overall UK is performing better then DE when it comes to paid_ltv/cac but I think it's important to factor the two platforms. I would recommend increasing spend for Android in both countries since the paid_ltv/cac is 1.05 for DE and 1.30 for UK and decreasing spend for iOS since the paid_ltv/cac is below 1.0 for both countries.


In [16]:
q2_filter = table_one_df[
    (table_one_df['country'].isin(['DE', 'UK']))
    & (table_one_df['media_source']=='Channel B')]
q2_filter.round(2)

,month,platform,media_source,country,spend,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,cpi,cpt,paid_cac,paid_ltv/cac,install_to_trial_rate,trial_to_purchase_rate
12,May,android,Channel B,UK,63112.54,27622.0,2853.0,1519.34,54.12,82232.93,2.28,22.12,41.54,1.30,0.10,0.53
14,May,ios,Channel B,UK,43153.05,8109.0,1500.0,664.73,49.18,32691.92,5.32,28.77,64.92,0.76,0.18,0.44
16,May,android,Channel B,DE,33832.14,19695.0,1594.0,607.55,58.50,35542.32,1.72,21.22,55.69,1.05,0.08,0.38
30,May,ios,Channel B,DE,18922.42,4503.0,795.0,322.05,51.94,16726.31,4.20,23.80,58.76,0.88,0.18,0.41


In [37]:
q2_filter_grp = t1_add_calulated_fields(q2_filter.groupby(['media_source', 'country']).sum())
q2_filter_grp.round(2)

spend  installers  trial_starts  \
media_source country                                        
Channel B    DE        52754.56     24198.0        2389.0   
             UK       106265.59     35731.0        4353.0   

                      forecasted_purchasers    ltv  forcasted_ltv_rev   cpi  \
media_source country                                                          
Channel B    DE                      929.61  56.23           52268.63  2.18   
             UK                     2184.07  52.62          114924.85  2.97   

                        cpt  paid_cac  paid_ltv/cac  install_to_trial_rate  \
media_source country                                                         
Channel B    DE       22.08     56.75          0.99                   0.10   
             UK       24.41     48.65          1.08                   0.12   

                      trial_to_purchase_rate  
media_source country                          
Channel B    DE                         0.39  
             UK                         0.50

## Questions 3 <a id='q3'></a>

**Which platform has a better trial conversion rate? Purchase conversion rate? Why do you think that is?**

Overall, iOS has a better trial conversion rate and a better purchase conversion rate. There's most likely multiple factors that can lead this kind of difference in conversion rate. After doing some more research here are the top factors why I believe iOS has better conversion rates:

##### Income Demographics
A [survey](https://nypost.com/2018/10/25/iphone-and-android-users-are-completely-different-people/) conducted in Oct 2018 compared a couple of metrics for iOS and Android users. The survey found that ios users had a higher average salary and, not surprisingly, also spent more money monthly.

##### App Store Rating
The [Apple App Store](https://apps.apple.com/us/app/calm/id571800810) has Calm rated at 4.8 while the [Google Play Store](https://play.google.com/store/apps/details?id=com.calm.android&hl=en_US&gl=US) only has 4.2.
   
##### Brand Loyalty
A [study](https://www.sellcell.com/blog/cell-phone-brand-loyalty-2021/) done in 2020 looked at brand loyalty and found that "Apple is at an all-time high of nearly 92%".


##### Shopping Experience
There is no denying that Apple has a more integrated ecosystem that translates to not only a better in app experience but also a better experience when making purchases. 

To conclude, I came across this [Mobile App Trend Report](https://liftoff.io/wp-content/uploads/2019/10/2019-Mobile-App-Trends-Report.pdf) that aligns with the possible factors above along with the data itself. Specifically on page 8 and 9 you'll see that even though Android users have more app downloads iOS has more purchases - which is exactly what we see with the data here. Android has a better CPI of \\$0.92 but performs worse on paid CAC compared to iOS.

In [18]:
q3_filter_grp = t1_add_calulated_fields(table_one_df.groupby(['platform']).sum())
q3_filter_grp.round(2)

,spend,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,cpi,cpt,paid_cac,paid_ltv/cac,install_to_trial_rate,trial_to_purchase_rate
platform,,,,,,,,,,,,
android,1092807.89,780811.77,43214.47,15865.99,70.51,1118663.73,1.40,25.29,68.88,1.02,0.06,0.37
ios,3052324.78,691300.73,93677.11,45371.67,74.66,3387413.05,4.42,32.58,67.27,1.11,0.14,0.48


## Questions 4 <a id='q4'></a>

**Based on your table, on which media sources and/or countries do you see an opportunity to push/invest further into? Why? Please provide 3 recommendations.**

My goal here was to find a combination country and media source that have already shown positive ROI but have not been fully tested on every media channel. Here are my top 3 recommendations in order of priority.
1. CL - Channel E
2. NO - Channel D
3. PE - Channel E

With the analysis conducted below I found that these three country and media source combinations fulfilled two criteria's that lead me to believe that they are strong opportunities to test in:
1. The country had a use base that was already established because they were in the top 25% of countries by installers.
    1. I had initially look at just the top 10% but I was only able to identify 2 that fell into all the criteria's.
2. The country was already tested and proven to perform on 4 media channels because they all had a paid_ltv/cac over 1.0.
    2. Therefore they all had 1 media channel that had not been tested or had very low spend on it.

In [43]:
# create q4_filter_df
q4_filter_df = table_one_df
# get list of top 25% countries by installers
q4_country_df = t1_add_calulated_fields(table_one_df.groupby('country').sum()).reset_index()
percentile_75_countries_by_install = q4_country_df[q4_country_df['installers'] > np.percentile(q4_country_df['installers'], 75)]['country'].to_list()

# filter for only top 25% of countries by installs
q4_2_filter_df = q4_filter_df[q4_filter_df['country'].isin(percentile_75_countries_by_install)]
q4_2_filter_grp = t1_add_calulated_fields(q4_2_filter_df.groupby(['country', 'media_source']).sum())

# sort paid_ltv/cac with highest on top
q4_2_filter_df = q4_2_filter_grp.sort_values('paid_ltv/cac', ascending=False).reset_index()

# filter out missing paid_ltv/cac and rows with 0 spend
q4_3_filter_df = q4_2_filter_df[
    q4_2_filter_df['paid_ltv/cac'].notna() & q4_2_filter_df['spend'] != 0
]

# groupby country and aggregate
q4_4_filter_df = q4_3_filter_df.groupby('country').agg(
    {'media_source': lambda x: x.nunique(), 
     'paid_ltv/cac': ['mean', lambda x: all(y > 1 for y in list(x))]
    }
).reset_index()

# rename columns for readability
q4_4_filter_df.columns = ['country', 'unq_media_source', 'avg_paid_ltv/cac', 'good_roi_on_all_channels']

# sort by number of unq media sources and avg_paid_ltv/cac
q4_5_filter_df = q4_4_filter_df.sort_values(
    ['unq_media_source', 'avg_paid_ltv/cac'], 
    ascending=False, 
    ignore_index=True
)

# filter for only countries that are missing a media source and are doing well on every other media source
q4_5_filter_df = q4_5_filter_df[
    (q4_5_filter_df['unq_media_source'] == 4) 
    & q4_5_filter_df['good_roi_on_all_channels']
]

q4_5_filter_df.round(2)

,country,unq_media_source,avg_paid_ltv/cac,good_roi_on_all_channels
7,CL,4,2.70,True
10,NO,4,2.56,True
11,PE,4,2.23,True


In [54]:
q4_5_filter_grp = t1_add_calulated_fields(
    q4_2_filter_df[q4_2_filter_df['country'].isin(q4_5_filter_df['country'])].groupby(['country', 'media_source']
                          ).sum())
q4_5_filter_grp[['spend', 'installers', 'forecasted_purchasers', 'ltv', 'forcasted_ltv_rev', 'paid_cac', 'paid_ltv/cac']].round(2)

spend  installers  forecasted_purchasers    ltv  \
country media_source                                                       
CL      Channel A     27556.23     27484.0                 641.26  45.77   
        Channel B      3737.57      5885.0                  95.72  45.79   
        Channel C       654.40       704.0                  64.30  53.28   
        Channel D        17.97        21.0                   1.13  53.15   
NO      Channel A      6453.40      1456.0                 123.13  57.72   
        Channel B      1361.31       584.0                  27.96  57.83   
        Channel C       282.05       195.0                  25.93  64.13   
        Channel D         5.10         6.0                   0.00    NaN   
        Channel E       302.80       138.0                  10.16  61.04   
PE      Channel A      7112.23      9220.0                 177.84  46.76   
        Channel B      1682.61      3233.0                  38.20  45.28   
        Channel C       382.50       418.0                  32.58  52.07   
        Channel D        27.00        27.0                   1.10  56.22   

                      forcasted_ltv_rev  paid_cac  paid_ltv/cac  
country media_source                                             
CL      Channel A              29350.60     42.97          1.07  
        Channel B               4382.60     39.05          1.17  
        Channel C               3425.93     10.18          5.24  
        Channel D                 60.02     15.92          3.34  
NO      Channel A               7107.30     52.41          1.10  
        Channel B               1617.08     48.69          1.19  
        Channel C               1662.90     10.88          5.90  
        Channel D                  0.00       inf           NaN  
        Channel E                619.97     29.81          2.05  
PE      Channel A               8316.06     39.99          1.17  
        Channel B               1729.52     44.05          1.03  
        Channel C               1696.42     11.74          4.44  
        Channel D                 61.85     24.54          2.29

## Questions 5 <a id='q5'></a>

**Based on your table, on which media sources and/or countries should media buyers constrain bids or budgets? Why? Please provide 3 recommendations.**

My goal here was to find the country and media source combinations that was losing the most money. Here are my top three recommendations in order of priority:

1. US - Channel A
2. US - Channel B
3. UK - Channel A

First I added a forecasted_profit column that I calculated as follows: 

$forecasted\_profit = forcasted\_ltv\_rev - spend$

I then sorted for profit lowest to highest and took the top three. There are 164 country and media source combinations in total that are losing money, but it's important to consider sample size in a situation like this. For example, we don't want to remove a country and media source combination when it hasn't has the opportunity to be tested over a longer period of time and with more then \$57.60 spend on it, as is the case with, IS/Channel D, the lowest profit loss combo.

In [21]:
# group by country, media_source
q5_filter_df = t1_add_calulated_fields(table_one_df.groupby(['country', 'media_source']).sum()).reset_index()

# add revenue column
q5_filter_df['forecasted_profit'] = q5_filter_df['forcasted_ltv_rev'] - q5_filter_df['spend']

# sort paid_ltv/cac with highest on top
q5_1_filter_df = q5_filter_df[q5_filter_df['paid_ltv/cac'] < 1].sort_values('forecasted_profit', ignore_index=True)

q5_1_filter_df.round(2)

,country,media_source,spend,installers,trial_starts,forecasted_purchasers,ltv,forcasted_ltv_rev,cpi,cpt,paid_cac,paid_ltv/cac,install_to_trial_rate,trial_to_purchase_rate,forecasted_profit
0,US,Channel A,1342664.48,198575.38,27583.92,12679.51,92.22,1169322.69,6.76,48.68,105.89,0.87,0.14,0.46,-173341.79
1,US,Channel B,602128.30,167524.91,15029.30,5829.39,92.74,540590.88,3.59,40.06,103.29,0.90,0.09,0.39,-61537.42
2,UK,Channel A,358885.13,61321.00,10984.00,5841.04,51.38,300098.23,5.85,32.67,61.44,0.84,0.18,0.53,-58786.90
3,BR,Channel A,130134.71,131270.00,6879.00,2210.24,48.02,106126.88,0.99,18.92,58.88,0.82,0.05,0.32,-24007.83
4,DE,Channel A,116981.13,29984.00,4792.00,1835.99,54.92,100829.56,3.90,24.41,63.72,0.86,0.16,0.38,-16151.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,GY,Channel A,66.67,46.00,1.00,1.10,48.60,53.47,1.45,66.67,60.60,0.80,0.02,1.10,-13.20
161,MU,Channel A,418.80,517.00,19.00,6.77,60.71,410.76,0.81,22.04,61.90,0.98,0.04,0.36,-8.04
162,SO,Channel A,83.21,66.00,2.00,1.11,70.76,78.27,1.26,41.60,75.23,0.94,0.03,0.55,-4.94
163,LB,Channel B,78.82,235.00,7.00,1.14,67.39,76.75,0.34,11.26,69.21,0.97,0.03,0.16,-2.07


## Questions 6 <a id='q6'></a>

**Which media source is performing the best? Which metrics are considering and why?**

In terms of profit generated Channel C takes the lead by far with a profit that is more then all the other channels combined. The more important metric to look at would be paid_ltv/cac this will allow me to see how efficient the media sources are. For this dataset, I would say Channel C is the best performing source overall.

In [55]:
# group by country, media_source
q6_filter_df = t1_add_calulated_fields(table_one_df.groupby('media_source').sum()).reset_index()

# add revenue column
q6_filter_df['rounded_forecasted_profit'] = round(q6_filter_df['forcasted_ltv_rev'] - q6_filter_df['spend'], 2)

# sort paid_ltv/cac with highest on top
q6_1_filter_df = q6_filter_df.sort_values('rounded_forecasted_profit', ascending=False, ignore_index=True)

q6_1_filter_df[['media_source', 'paid_ltv/cac', 'rounded_forecasted_profit']]

,media_source,paid_ltv/cac,rounded_forecasted_profit
0,Channel C,2.997133,757883.10
1,Channel D,1.010951,98.11
2,Channel E,0.889951,-8894.39
3,Channel B,0.973317,-26819.81
4,Channel A,0.864710,-361322.90


## Table 2 <a id='t2'></a>

Table two requires a different aggregation then table 1. Since metrics need to be created for paid and organic traffic types I'll need to do further cleaning. You'll notice I've left in a couple of additional helper columns that weren't asked for in table 2. In the dataset file I've highlighted the columns that were requested green all other tabs are helper tabs that provide more information. Below is the first 5 rows of table 2.

Please refer to the "table_2_dataset" tab in the "all_datasets.xlsx" file I shared with you to see table 2 in its entirety.


In [23]:
prep_table_two_df = merged_df.groupby(['month', 'country', 'platform', 'traffic_type'], dropna=False).sum().reset_index()

paid_merged_df = prep_table_two_df[prep_table_two_df['traffic_type'] == 'paid']
organic_merged_df = prep_table_two_df[prep_table_two_df['traffic_type'] == 'organic']


table_two_df = paid_merged_df.merge(
    right=organic_merged_df,
    how='outer',
    on=['month', 'country', 'platform'],
    suffixes=('_paid', '_org'),
    validate='one_to_one',
    indicator=True
)

# merge columns that don't need to be seprated into organic and non-organic
table_two_df['spend'] = table_two_df['spend_paid'] + table_two_df['spend_org']
table_two_df['installers'] = table_two_df['installers_paid'] + table_two_df['installers_org']
table_two_df['trial_starts'] = table_two_df['trial_starts_paid'] + table_two_df['trial_starts_org']

# filter for relevant headers
table_two_df = table_two_df[
    ['month', 'country', 'platform', 'spend', 'installers', 'trial_starts', 
     'forecasted_purchasers_paid', 'forecasted_purchasers_org', 
     'forcasted_ltv_rev_paid', 'forcasted_ltv_rev_org']
]

# create function to add calculated fields
def t2_add_calulated_fields(df):
    df['forecasted_blended_purchases'] = df['forecasted_purchasers_paid'] + df['forecasted_purchasers_org']
    df['blended_forcasted_ltv_rev'] = df['forcasted_ltv_rev_paid'] + df['forcasted_ltv_rev_org']
    df['avg_paid_ltv'] = df['forcasted_ltv_rev_paid']/df['forecasted_purchasers_paid']
    df['avg_org_ltv'] = df['forcasted_ltv_rev_org']/ df['forecasted_purchasers_org']
    df['avg_blended_ltv'] = df['blended_forcasted_ltv_rev']/df['forecasted_blended_purchases']
    df['paid_cac'] = df['spend'] / df['forecasted_purchasers_paid']
    df['blended_cac'] = df['spend'] / df['forecasted_blended_purchases']
    df['paid_ltv/cac'] = df['avg_paid_ltv'] / df['paid_cac']
    df['blended_ltv/cac'] = df['avg_blended_ltv'] / df['blended_cac']
    df['organic_purchase_mix'] = df['forecasted_purchasers_org'] / df['forecasted_blended_purchases']
    return df

# sort by country
table_two_df.sort_values('country', ascending=False, ignore_index=True)

# add table 2 calulated fields
table_two_df = t2_add_calulated_fields(table_two_df)

# show first 5 rows
table_two_df.head().round(2)

,month,country,platform,spend,installers,trial_starts,forecasted_purchasers_paid,forecasted_purchasers_org,forcasted_ltv_rev_paid,forcasted_ltv_rev_org,forecasted_blended_purchases,blended_forcasted_ltv_rev,avg_paid_ltv,avg_org_ltv,avg_blended_ltv,paid_cac,blended_cac,paid_ltv/cac,blended_ltv/cac,organic_purchase_mix
0,May,AD,android,25.47,168.0,4.0,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,inf,inf,NaN,NaN,NaN
1,May,AD,ios,111.06,88.0,3.0,1.18,1.14,55.23,70.74,2.32,125.97,46.70,61.97,54.20,93.90,47.78,0.5,1.13,0.49
2,May,AE,android,987.64,1808.0,125.0,21.51,31.65,1477.21,2611.79,53.16,4089.00,68.67,82.52,76.92,45.91,18.58,1.5,4.14,0.60
3,May,AE,ios,9677.55,4561.0,576.0,148.61,85.50,9685.32,5873.69,234.11,15559.01,65.17,68.70,66.46,65.12,41.34,1.0,1.61,0.37
4,May,AF,android,13.66,89.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN,inf,inf,NaN,NaN,NaN


## Questions 7 <a id='q7'></a>

**Which platform has a higher LTV? Why might that be?**

iOS has a higher avg_blended_ltv. I believe the LTV is better on iOS for the same reasons I outlined in [Question 3](#q3).

In [38]:
# create q4_filter_df
q7_filter_df = t2_add_calulated_fields(table_two_df.groupby('platform').sum())

# show summary by platform
q7_filter_df[['spend', 'avg_paid_ltv', 'avg_org_ltv', 'avg_blended_ltv']].round(2)

,spend,avg_paid_ltv,avg_org_ltv,avg_blended_ltv
platform,,,,
android,1092519.83,70.51,76.96,73.06
ios,3050212.50,74.66,80.25,77.41


## Questions 8 <a id='q8'></a>

**On iOS, which of the top 10 highest spend countries has the worst organic mix? Why might that be?**

Below are you top 10 countries by spend sorted from least to greatest by organic_purchase_mix. Brazil has the worst organic_purchase_mix in this list. I researched recent news regarding Brazil and iOS, and in March 2021 the [Brazil fined Apple \\$2 Million](https://www.entrepreneur.com/article/367660) for not providing a charger with the new iPhone. It is possible that this news diminished the iOS brand reputation for Brazil.

In [39]:
# create q4_filter_df
q8_filter_df = table_two_df[table_two_df['platform']=='ios']

# sort by country
q8_filter_df = q8_filter_df.sort_values('spend', ascending=False, ignore_index=True).head(10)

# sort by organic_purchase_mix
q8_filter_df = q8_filter_df.sort_values('organic_purchase_mix', ignore_index=True)

# show top 10
q8_filter_df[['country', 'platform', 'spend', 'forecasted_purchasers_paid', 'forecasted_purchasers_org', 'forecasted_blended_purchases', 'organic_purchase_mix']].round(2)

,country,platform,spend,forecasted_purchasers_paid,forecasted_purchasers_org,forecasted_blended_purchases,organic_purchase_mix
0,BR,ios,139375.98,2341.79,557.37,2899.16,0.19
1,MX,ios,45493.85,928.39,450.17,1378.56,0.33
2,CH,ios,30796.74,518.41,392.83,911.24,0.43
3,KR,ios,31235.09,559.12,455.87,1014.98,0.45
4,UK,ios,379565.21,8139.29,6673.49,14812.78,0.45
5,US,ios,1738884.52,21247.53,22756.81,44004.35,0.52
6,FR,ios,33754.16,534.33,574.57,1108.90,0.52
7,CA,ios,123434.12,2042.29,2730.48,4772.77,0.57
8,DE,ios,107780.73,1918.47,2698.41,4616.87,0.58
9,AU,ios,96641.59,1624.10,2318.40,3942.50,0.59


## Questions 9 <a id='q9'></a>

**Why would you want to consider both Paid and Blended LTV/CAC when deciding to push ad spend in a particular platform or country?**

I've listed the main reasons to factor in both Paid and Blended LTV/CAC in to your analysis:

- **Cost**: Organic marketing is cheaper because you don't need to dedicate as much time or resources.
- **Lifetime Value**: When comparing the LTV for organic vs non-organic we can see that Organic LTV is higher which makes sense since it yields more authentic customer engagement.
- **Brand Awareness**: Organic Marketing allows for users to discover Calm passively and amplify the effectiveness of Paid marketing.

In [26]:
weighted_paid_ltv = (table_two_df['forcasted_ltv_rev_paid'].sum() / table_two_df['forecasted_purchasers_paid'].sum()).round(2)
weighted_org_ltv = (table_two_df['forcasted_ltv_rev_org'].sum() / table_two_df['forecasted_purchasers_org'].sum()).round(2)

print(f'Weighted Non-Organic LTV: {weighted_paid_ltv}')
print(f'Weighted Organic LTV: {weighted_org_ltv}')

Weighted Non-Organic LTV: 73.58
Weighted Organic LTV: 79.62
